In [1]:
import csv as csv
import numpy as np

In [2]:
csv_file_object = csv.reader(open('train.csv','rb'))

In [3]:
header=csv_file_object.next()

In [16]:
header

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [4]:
data =[]

In [12]:
for row in csv_file_object:
    data.append(row)

In [13]:
len(data)

891

In [14]:
data=np.array(data)

In [15]:
type(data)

numpy.ndarray

In [19]:
# If the fare is greater than 39, then change the data in that field
# to 39 only

fare_ceiling =40

data[data[:,9].astype(np.float)>= fare_ceiling,9]=fare_ceiling -1

# print data[:,9]

In [21]:
# create the price buckets
# Calculate the different classes on board. ALthough we know that value is 3

fare_bucket_size=10

number_of_price_buckets = fare_ceiling/fare_bucket_size

number_of_classes = len(np.unique(data[:,2]))

In [25]:
# This reference matrix will show the proportion of survivors as a sorted table of
# gender, class and ticket fare.
# First initialize it with all zeros
# 2 rows corresponding to men and women

survival_table = np.zeros([2,number_of_classes,number_of_price_buckets],float)

In [33]:
survival_table.shape

(2, 3, 4)

In [41]:
# Stats of all women and men on the board

for i in xrange(number_of_classes):
    for j in xrange(number_of_price_buckets):
        
        women_only_stats = data[(data[:,4]=='female')\
                               & (data[:,2].astype(np.float)==i+1)\
                               & (data[:,9].astype(np.float)>= j*fare_bucket_size)\
                               & (data[:,9].astype(np.float)< (j+1)*fare_bucket_size),1]
        
        men_only_stats = data[(data[:,4]!='female')\
                               & (data[:,2].astype(np.float)==i+1)\
                               & (data[:,9].astype(np.float)>= j*fare_bucket_size)\
                               & (data[:,9].astype(np.float)< (j+1)*fare_bucket_size),1]
        
    
        # for women
        survival_table[0,i,j]= np.mean(women_only_stats.astype(np.float))
        # for men
        survival_table[1,i,j]=np.mean(men_only_stats.astype(np.float))

In [42]:
print survival_table

[[[        nan         nan  0.83333333  0.97727273]
  [        nan  0.91428571  0.9         1.        ]
  [ 0.59375     0.58139535  0.33333333  0.125     ]]

 [[ 0.                 nan  0.4         0.38372093]
  [ 0.          0.15873016  0.16        0.21428571]
  [ 0.11153846  0.23684211  0.125       0.24      ]]]


In [48]:
# Since in python if it tries to find the mean of an array with nothing in it
# (such that the denominator is 0), then it returns nan, we can convert these to 0
# by just saying where does the array not equal the array, and set these to 0.

survival_table[survival_table!= survival_table]=0
print survival_table

[[[ 0.          0.          0.83333333  0.97727273]
  [ 0.          0.91428571  0.9         1.        ]
  [ 0.59375     0.58139535  0.33333333  0.125     ]]

 [[ 0.          0.          0.4         0.38372093]
  [ 0.          0.15873016  0.16        0.21428571]
  [ 0.11153846  0.23684211  0.125       0.24      ]]]


In [49]:
# Now I have my proportion of survivors, simply round them such that if <0.5
# I predict they dont surivive, and if >= 0.5 they do

survival_table[survival_table <0.5]=0
survival_table[survival_table >0.5]=1

In [50]:
print survival_table

[[[ 0.  0.  1.  1.]
  [ 0.  1.  1.  1.]
  [ 1.  1.  0.  0.]]

 [[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]]


In [63]:
# Now I have my indicator I can read in the test file and write out
# if a women then survived(1) if a man then did not survived (0)
# First read in test
test_file = open('test.csv', 'rb')
test_file_object = csv.reader(test_file)
header = test_file_object.next()

# Also open the a new file so I can write to it. 
predictions_file = open("genderclassmodel_rg.csv", "wb")
predictions_file_object = csv.writer(predictions_file)
predictions_file_object.writerow(["PassengerId", "Survived"])

In [64]:


print header
print number_of_classes
print number_of_price_buckets

['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
3
4


In [65]:
# first thing to do is bin up the price file
for row in test_file_object:
    for j in xrange(number_of_price_buckets):
        try:
            row[8] = float(row[8]) # No fare will be come up as string
            
        except:
            bin_fare = 3-float(row[1]) # # If there is no fare then place the price of the ticket according to class
            break
            
        if row[8]>fare_ceiling:
            bin_fare= number_of_price_buckets -1
            break
        
        if row[8]>= j*fare_bucket_size and row[8]<(j+1)*fare_bucket_size:
            bin_fare =j
            break
            
    if row[3]=='female':
        predictions_file_object.writerow([row[0],"%d" % int(survival_table[0,float(row[1])-1,bin_fare])])
        
    else:
        predictions_file_object.writerow([row[0],"%d" % int(survival_table[1,float(row[1])-1,bin_fare])])
            
            
            
            
            

/Users/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:23: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:20: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [66]:
test_file.close()
predictions_file.close()